# Project - Apache Spark & Elastichsearch

##### Students:
* Lilia IZRI      (DS)
* Yacine MOKHTARI (DS)
* Alexandre COMBEAU (DS)

##### Report
[PENSER A METTRE UN LIEN ICI]


In [2]:
# pip install tweepy

In [3]:
# Imports
import tweepy
from tweepy.auth import OAuthHandler
from tweepy import Stream
import socket
import json
from utils import get_location_by_address

## I. Processing Twitter as a Stream

### 1. Settings : Insert Twitter developer credentials

In [4]:
# KEYS (Remarque mdr c'est des clés que j'ai trouvées sur un site 🤷🏻‍♀️)
consumer_key    = '3XwgFgsXucaFXOrkZtlwvxV5O'
consumer_secret = 'LNnTcxibKalylVImljDsKfkRqb6WHD8I7hdSYt7Pm7VaCetqZm'
access_token    = '608048932-AM21gcwLORlm5j0514DwPL850byLb7Gs04bPZ6Mn'
access_secret   = 'QtnJ8mV75peV7L7Kd4DZKogMRIxzjk5XFOPtZiz62NoM3'

### 2. Creating a new listener class

In [5]:
# Class that represents our Listener
class TweetsListener(Stream):
    
    def __init__(self, *args, csocket):
        super().__init__(*args)
        self.client_socket = csocket
    def on_data(self, data):
        """
        Here, we extract these fields : 
            - .... 
        """
        try:
            ### Extracting interesting fields (text, username and date)
            tweet = json.loads( data )
            text = tweet['text']
            user = tweet['user']['screen_name']
            date = tweet['created_at']
            
            ### Handle hashtags (we join the list by adding '#' and separate them by a space ' '
            list_hashtags = tweet['entities']['hashtags']
            hashtags = " ".join(['#' + i['text' ] for i in list_hashtags]) if list_hashtags is not [] else ""
            
            ### Handle the location 
            location = get_location_by_address(tweet['user']['location']) 
            latitude, longitude = location if location is not None else get_location_by_address('Unknown')
            print(latitude, longitude)
            
            ###  We combine data and metadata to send them 
            #  We add a key ###:field:### so we can split the fields easily 
            #  We remove '\n' from a tweet and put one '\n' between tweets 
            tweet_info = (" ###:field:### user: " + user      + " ###:field:### tweet: "    + text +\
                          " ###:field:### date: " + date      + " ###:field:### lat: "      + latitude + \
                          " ###:field:### lon: "  + longitude + " ###:field:### hashtags: " + hashtags + \
                          " ###:field:### id: "+ tweet['user']['id_str']
                          ).replace('\n', ' ') + '\n'
            
            ### Uncomment to see the string we pass through the TCP port
            # print(tweet_info)
            
            ### Send to socket : We convert this tweet into a bite code (since spark takes easily this kind of data)
            self.client_socket.send(tweet_info.encode('utf-8'))
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

    # Handle errors
    def if_error(self, status):
        print(status)
        return True

The method that captures these tweets in spark (```socketTextStream(port..)```) will concider a text containing a single "\n" as two different entries. <br>
Since we don't want the same tweet to be separated in two differents RDDs, we replace each '\n' by a simple space ' '

###  3. Authenticate our connection using the pre-defined credentials 
In order to begin streaming, we have to implement a function that allows us to authenticate our connection

In [6]:
def send_tweets(c_socket):
    """
        Send tweets to client socket (in our case ✨SPARK✨)
    """
    twtr_stream = TweetsListener(
        consumer_key, consumer_secret,
        access_token, access_secret,
        csocket=c_socket
    )
    twtr_stream.filter(track=['depp'])#, 'depp', 'macron', 'covid', 'ukraine', 'heard', 'yoda', 'thread', 'sport'])

###  4. Testing connection with client (Spark)
The second step would be to pass the collected tweets to Spark. A way to do that, is to send them in a TCP/socket (that would be the entry point of Spark Streaming)

In [7]:
new_skt = socket.socket()         # initiate a socket object
host = "127.0.0.1"                # address host
port = 5567                       # specifie port 
new_skt.bind((host, port))        # Binding host and port

In [ ]:
print(f"Now listening on port: {port}")
new_skt.listen(5)                 # waiting for client connection
c, addr = new_skt.accept()        # Establish connection with client

print(f"Received request from: {addr}")
send_tweets(c)                    # send tweets to client socket

Now listening on port: 5567
Received request from: ('127.0.0.1', 44140)
44.933143 7.540121
42.4494141 27.2936714
-22.9110137 -43.2093727
54.7023545 -3.2765753
44.933143 7.540121
-12.0621065 -77.0365256
44.933143 7.540121
44.933143 7.540121
53.1908873 -2.8908955
-12.544455500000002 131.0183706011867
44.933143 7.540121
35.3258295 -80.6454979
27.7567667 -81.4639835
4.5693754 102.2656823
-23.3165935 -58.1693445
44.933143 7.540121
19.37218875 -72.33465445105234
51.47091115 -2.5790334025708415
30.2711286 -97.7436995
44.933143 7.540121
32.7174202 -117.1627728
44.933143 7.540121
32.2228765 -110.9748477
44.933143 7.540121
10.506098 -66.9146017
44.933143 7.540121
37.0412584 50.0068157
44.933143 7.540121
53.1908873 -2.8908955
44.933143 7.540121
25.029422 -77.36195598496681
25.029422 -77.36195598496681
51.5073219 -0.1276474
44.933143 7.540121
44.933143 7.540121
44.933143 7.540121
44.933143 7.540121
-34.9058916 -56.1913095
44.933143 7.540121
44.933143 7.540121
36.7014631 -118.755997
44.933143 7.540